# **Parties in Power (PiP) Master Notebook**

---

**Created**: 15/03/2025  
**Purpose**: To create visualisations for the PiP article **(ADD LINK HERE)**  
**Prerequisites**: Data preparation notebooks to be run for each country **(ADD LINK HERE)**  
**Output**: JPEG-format visualisations for: 1) Change in political alignment by popular vote; 2) Change in political alignment by party in power.

---

## **Import Required Libraries**

In [1]:
# Import pandas
import pandas as pd

# Import matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Import os to access directories
import os

# Import json for config files
import json

## **Import Processed Data**

### **Define Folder Path for Processed Data**

In [18]:
# Define main directory & config filepath
main_directory = os.path.abspath(os.path.join(os.getcwd(), '..\..'))  # Go up one level to the main directory

# Define folder path for processed election data
inp_path = os.path.join(main_directory, '03. Output\\01. Intermediate\\')

### **Create Dictionary of Election Results for Each Country**

In [19]:
# Get all CSV files in the directory
csv_files = [f for f in os.listdir(inp_path) if f.endswith('.csv')]

# Dictionary to store paired DataFrames
country_dict = {}

# Iterate through files and store them in dictionary
for file in csv_files:
    country, version = file.rsplit("_", 1)  # Split into country and version (e.g., "UK", "Prev.csv")
    version = version.replace(".csv", "")  # Remove ".csv"

    # Read CSV into DataFrame
    df = pd.read_csv(f"{inp_path}\{file}")

    # Initialize country key if not exists
    if country not in country_dict:
        country_dict[country] = {}

    # Store the DataFrame under "Prev" or "New"
    country_dict[country][version] = df

## **Create Master Dataframe**

### **Join All Dataframes**

In [24]:
# Initialise list of rows for master_df
master_lst = []

# Loop through all dfs
for country, df_dict in country_dict.items():
    # Extract prev and new dfs
    prev_df = df_dict["Prev"]
    new_df = df_dict["New"]

    # Order dfs by Vote Prop
    prev_df = prev_df.sort_values("In-Scope Vote Prop", ascending=False).reset_index(drop=True)
    new_df = new_df.sort_values("In-Scope Vote Prop", ascending=False).reset_index(drop=True)

    # Compute Political Spectrum Scores (dot product of Alignment and In-Scope Vote Prop)
    prev_score = prev_df["Alignment Score"].dot(prev_df["In-Scope Vote Prop"])
    new_score = new_df["Alignment Score"].dot(new_df["In-Scope Vote Prop"])

    # Get alignment of majority party for each election
    prev_party = prev_df["Alignment Score"][0]
    new_party = new_df["Alignment Score"][0]
    
    # Append the results as a new row in the master list
    master_lst.append({
        "Country": country, 
        "Previous Political Spectrum Score": prev_score, 
        "Latest Political Spectrum Score": new_score,
        "Previous Majority Party": prev_party,
        "Latest Majority Party": new_party
    })

# Create master DataFrame
master_df = pd.DataFrame(master_lst)

### **Create Alignment Change Column**

In [25]:
# Create alignment change col
master_df["Alignment Change"] = master_df["Latest Political Spectrum Score"] - master_df["Previous Political Spectrum Score"]

# Sort by biggest change (from left to right) asc
master_df = master_df.sort_values("Latest Political Spectrum Score", ascending=True).reset_index(drop=True)

# Convert 'Country' column to categorical for correct plotting order
master_df['Country'] = pd.Categorical(master_df['Country'], categories=master_df['Country'], ordered=True)

# Display the final master DataFrame
master_df

,Country,Previous Political Spectrum Score,Latest Political Spectrum Score,Previous Majority Party,Latest Majority Party,Alignment Change
0,UK,0.430000,0.070000,3,-2,-0.360000
1,Canada,0.084484,0.232640,-1,-1,0.148156
2,Belgium,0.207070,0.255894,2,2,0.048825
3,Australia,0.759323,0.346381,3,3,-0.412942
4,US,0.392500,0.517513,-2,3,0.125013
5,Sweden,0.608662,0.693772,-2,-2,0.085110
6,Italy,-0.484269,0.830414,-4,5,1.314683
7,Germany,0.628578,0.946423,2,2,0.317845
8,Netherlands,1.125280,1.469460,2,5,0.344180
9,France,0.544989,1.535629,1,6,0.990640


## **Create Visualisations**

### **Change in Political Alignment by Popular Vote**